# Data Process

In [ ]:
import json
import pandas as pd

In [ ]:
f = open("Data/2005_codes.json",encoding='utf-8')
json2005=json.load(f)

In [ ]:
data = None
for i in range(2005,2016):
    if i != 2012:
        d1 = pd.read_csv("Data/%d_data.csv" %i,usecols=[6,7,16,23,24,25,26,27])
        data = pd.concat([data, d1],axis=0).reset_index(drop = True)
    else:
        d2 = pd.read_csv("Data/%d_data.csv" %i,usecols=[6,7,16,23,24,25,26,27]).rename(columns={"icd_code_10":"icd_code_10th_revision"})
        data = pd.concat([data, d2],axis=0).reset_index(drop = True)

In [ ]:
data.head()

,detail_age_type,detail_age,current_data_year,icd_code_10th_revision,358_cause_recode,113_cause_recode,130_infant_cause_recode,39_cause_recode
0,1,45,2005,C439,98,28,NaN,15
1,1,61,2005,J439,266,84,NaN,28
2,1,79,2005,I698,239,70,NaN,24
3,1,50,2005,E119,159,46,NaN,16
4,1,68,2005,C349,93,27,NaN,8


In [ ]:
d_counts = pd.DataFrame.from_dict(dict(data["358_cause_recode"].value_counts()),orient='index',columns = ["Times"]).reset_index().rename(columns = {"index":"358_code"})
d_counts.head()

,358_code,Times
0,215,2212520
1,93,1733447
2,211,1392415
3,267,1367934
4,175,1164777


In [ ]:
def to_358(c1):
    return json2005['358_cause_recode'][str(c1).zfill(3)]
d_counts["358_causes"] = d_counts["358_code"].apply(to_358)

In [ ]:
def to_113(c1):
    code_113 = data[data["358_cause_recode"] == c1].reset_index().loc[0,:]["113_cause_recode"]
    return json2005['113_cause_recode'][str(code_113).zfill(3)]
d_counts["113_causes"] = d_counts["358_code"].apply(to_113)

In [ ]:
def to_39(c1):
    code_39 = data[data["358_cause_recode"] == c1].reset_index().loc[0,:]["39_cause_recode"]
    return json2005['39_cause_recode'][str(code_39).zfill(3)]
d_counts["39_causes"] = d_counts["358_code"].apply(to_39)

## rough causes


In [ ]:
def to_total(cause_39):
    r = 0
    list_39 = sorted(list(d_counts["39_causes"].unique()))
    
    if cause_39 in [list_39[0],list_39[1],list_39[2],list_39[4],list_39[7],list_39[16],list_39[27],list_39[34],list_39[35],list_39[36]]:
        r = "Others"
    elif cause_39 in [list_39[11],list_39[12]]:
        r = "3 highs"
    elif cause_39 in [list_39[13],list_39[15],list_39[37],list_39[38]]:
        r = "Infectious diseases"
    elif cause_39 in [list_39[14],list_39[17],list_39[31]]:
        r = "Heart diseases"
    elif cause_39 in list_39[19:27] or cause_39 in [list_39[29],list_39[32]]:
        r = "Malignant neoplasms"
    else:
        r = cause_39.split("(")[0].strip(" ")
    return r
d_counts["rough_causes"] = d_counts["39_causes"].apply(to_total)

In [ ]:
d_counts.to_excel("total.xlsx")

In [ ]:
causes_358 = {}
dict_h = dict(data["358_cause_recode"].value_counts())
for item in dict_h.keys():
    causes_358[json2005['358_cause_recode'][str(item).zfill(3)]] = dict_h[item]

In [ ]:
d_358 = pd.DataFrame.from_dict(causes_358,orient='index',columns = ["Times"]).reset_index()
d_358.head(10)

,index,Times
0,All other forms of chronic ischemic heart dise...,2212520
1,"Of trachea, bronchus and lung (C33-C34)",1733447
2,Acute myocardial infarction (I21-I22),1392415
3,Other chronic obstructive pulmonary disease (J44),1367934
4,"Organic dementia (F01,F03)",1164777
5,Alzheimer=s disease (G30),921265
6,Diabetes mellitus (E10-E14),807571
7,"Stroke, not specified as hemorrhage or infarct...",755369
8,"Atherosclerotic cardiovascular disease, so des...",657654
9,Congestive heart failure (I50.0),607764


In [ ]:
causes_39 = {}
dict_39 = dict(data["39_cause_recode"].value_counts())
for item in dict_39.keys():
    causes_39[json2005['39_cause_recode'][str(item).zfill(3)]] = dict_39[item]

In [ ]:
d_39 = pd.DataFrame.from_dict(causes_39,orient='index',columns = ["Times"]).reset_index()
d_39.head()

,index,Times
0,"All other diseases (Residual) (A00-A09,A20-A49...",4870661
1,Ischemic heart diseases (I20-I25),4306494
2,"Other diseases of heart (I00-I09,I26-I51)",2060821
3,"Malignant neoplasms of trachea, bronchus and l...",1733447
4,"Other malignant neoplasms (C00-C15,C17,C22-C24...",1630010


In [ ]:
dict_113 = dict(data["113_cause_recode"].value_counts())
causes_113 = {}
for item in dict_113.keys():
    causes_113[json2005['113_cause_recode'][str(item).zfill(3)]] = dict_113[item]
d_113 = pd.DataFrame.from_dict(causes_113,orient='index',columns = ["Times"]).reset_index()
d_113.head(10)

,index,Times
0,"All other diseases (Residual) (D65-E07,E15-E34...",3028258
1,All other forms of chronic ischemic heart dise...,2212520
2,"Malignant neoplasms of trachea, bronchus and l...",1733447
3,Cerebrovascular diseases (I60-I69),1471444
4,Acute myocardial infarction (I21-I22),1392415
5,"Other chronic lower respiratory diseases (J44,...",1378763
6,"All other forms of heart disease (I26-I28,I34-...",1327848
7,Alzheimer's disease (G30),921265
8,Diabetes mellitus (E10-E14),807571
9,All other and unspecified malignant neoplasms ...,719789


In [ ]:
d_113.to_excel("113.xlsx")